In [2]:
import torch

In [ ]:
torch.cuda.empty_cache()

: 

In [1]:
pip install pygame

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 660.6 kB/s eta 0:00:17
   ---------------------------------------- 0.1/10.8 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.2/10.8 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/10.8 MB 1.8 MB/s eta 0:00:06
   - -------------------------------------- 0.5/10.8 MB 2.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/10.8 MB 2.7 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/10.8 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.1/10.8 MB 2.9 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/10.8 MB 3.1 MB/s eta 0:00:04
   ----- ---------------------------------- 1.5/10.8 MB 3.3 MB/s eta 0:00:03
   ------ -----

In [1]:
##maybe final##

import time
import threading
import random
from collections import deque
import pygame

pygame.init()

screen_width = 1000
screen_height = 800
FPS = 120

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (169, 169, 169)
BLUE = (0, 0, 255)

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Traffic Management Simulation")

font = pygame.font.Font(None, 36)

def get_vehicle_count(signal):
    return random.randint(5, 50)

traffic_lights = {
    'signal_1': 'RED',
    'signal_2': 'RED',
    'signal_3': 'RED',
    'signal_4': 'RED'
}

vehicle_counts = {
    'signal_1': 0,
    'signal_2': 0,
    'signal_3': 0,
    'signal_4': 0
}

vehicles = []

class Vehicle:
    def __init__(self, x, y, lane, direction, signal):
        self.x = x
        self.y = y
        self.lane = lane
        self.direction = direction
        self.signal = signal
        self.speed = random.randint(2, 5)
    
    def can_move(self):
        return traffic_lights[self.signal] == 'GREEN'

    def move(self):
        if self.can_move():
            if self.direction == "right":
                self.x += self.speed
                if self.x > screen_width:
                    self.x = -100
            elif self.direction == "left":
                self.x -= self.speed
                if self.x < -100:
                    self.x = screen_width + 50
            elif self.direction == "up":
                self.y -= self.speed
                if self.y < -100:
                    self.y = screen_height + 50
            elif self.direction == "down":
                self.y += self.speed
                if self.y > screen_height:
                    self.y = -100

    def draw(self):
        car_body = pygame.Rect(self.x, self.y, 50, 30)
        pygame.draw.rect(screen, BLUE, car_body)
        pygame.draw.circle(screen, BLACK, (self.x + 10, self.y + 30), 5)
        pygame.draw.circle(screen, BLACK, (self.x + 40, self.y + 30), 5)

def add_vehicles():
    lanes = {
        'signal_1': (0, 200, 'right'),
        'signal_2': (screen_width, 300, 'left'),
        'signal_3': (400, screen_height, 'up'),
        'signal_4': (500, 0, 'down')
    }
    
    for signal, (x, y, direction) in lanes.items():
        for _ in range(vehicle_counts[signal]):
            vehicles.append(Vehicle(x, y, signal, direction, signal))

def update_vehicle_counts():
    for signal in vehicle_counts.keys():
        vehicle_counts[signal] = get_vehicle_count(signal)
    add_vehicles()

def open_signal(signal):
    for s in traffic_lights.keys():
        traffic_lights[s] = 'RED'
    traffic_lights[signal] = 'GREEN'
    print(f"Signal {signal} is GREEN")

def clear_vehicles(signal):
    global vehicles
    vehicles = [vehicle for vehicle in vehicles if vehicle.signal != signal]
    print(f"Clearing vehicles from {signal}")

def display_status():
    while True:
        screen.fill(WHITE)

        pygame.draw.rect(screen, GRAY, (0, 190, screen_width, 160))
        pygame.draw.rect(screen, GRAY, (390, 0, 160, screen_height))
    

        for x in range(0, screen_width, 40):
            pygame.draw.rect(screen, WHITE, (x, 265, 20, 5))

        for y in range(0, screen_height, 40):
            pygame.draw.rect(screen, WHITE, (465, y, 5, 20))
        
        pole_positions = [(50, 200), (750, 300), (400, 50), (500, 500)]
        signal_names = ['Signal 1', 'Signal 2', 'Signal 3', 'Signal 4']
        
        for idx, pos in enumerate(pole_positions):
            pygame.draw.rect(screen, BLACK, (pos[0], pos[1] - 50, 10, 60))

            current_signal = f'signal_{idx + 1}'
            if traffic_lights[current_signal] == 'RED':
                pygame.draw.circle(screen, RED, (pos[0] + 5, pos[1] - 45), 5)
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 25), 5)
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 5), 5)
            elif traffic_lights[current_signal] == 'GREEN':
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 45), 5)
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 25), 5)
                pygame.draw.circle(screen, GREEN, (pos[0] + 5, pos[1] - 5), 5)

            text_name = font.render(signal_names[idx], True, BLACK)
            screen.blit(text_name, (pos[0] - 10, pos[1] + 20))

        for idx, (signal, count) in enumerate(vehicle_counts.items()):
            text = font.render(f"{signal} Count: {count}", True, BLACK)
            screen.blit(text, (50, 20 + idx * 40))

            current_color = traffic_lights[signal]
            text_color = font.render(f"{current_color}", True, BLACK)
            screen.blit(text_color, (280, 20 + idx * 40))

        for vehicle in vehicles:
            vehicle.draw()
            vehicle.move()

        pygame.display.flip()
        time.sleep(0.1)

def manage_traffic():
    while True:
        update_vehicle_counts()
        highest_traffic_signal = max(vehicle_counts, key=vehicle_counts.get)
        
        open_signal(highest_traffic_signal)
        time.sleep(15)
        clear_vehicles(highest_traffic_signal)
        open_signal('none')
        time.sleep(10)

def start_traffic_management():
    traffic_management_thread = threading.Thread(target=manage_traffic)
    traffic_management_thread.start()

def start_display_status():
    status_thread = threading.Thread(target=display_status)
    status_thread.start()

if __name__ == "__main__":
    start_traffic_management()
    start_display_status()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.quit()

pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Signal signal_1 is GREEN
Clearing vehicles from signal_1
Signal none is GREEN


KeyboardInterrupt: 

Signal signal_4 is GREEN
Clearing vehicles from signal_4
Signal none is GREEN
Signal signal_4 is GREEN
Clearing vehicles from signal_4
Signal none is GREEN
Signal signal_2 is GREEN
Clearing vehicles from signal_2
Signal none is GREEN
Signal signal_4 is GREEN
Clearing vehicles from signal_4
Signal none is GREEN
Signal signal_4 is GREEN
Clearing vehicles from signal_4
Signal none is GREEN
Signal signal_3 is GREEN
Clearing vehicles from signal_3
Signal none is GREEN
Signal signal_3 is GREEN
Clearing vehicles from signal_3
Signal none is GREEN
Signal signal_1 is GREEN
Clearing vehicles from signal_1
Signal none is GREEN
Signal signal_1 is GREEN
Clearing vehicles from signal_1
Signal none is GREEN
Signal signal_1 is GREEN
Clearing vehicles from signal_1
Signal none is GREEN
Signal signal_4 is GREEN
Clearing vehicles from signal_4
Signal none is GREEN
Signal signal_3 is GREEN
Clearing vehicles from signal_3
Signal none is GREEN
Signal signal_4 is GREEN
Clearing vehicles from signal_4
Signal 

In [ ]:
import time
import threading
import random
import pygame

pygame.init()

# Screen settings
screen_width = 1000
screen_height = 800
FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (169, 169, 169)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)

# Initialize screen
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Enhanced Traffic Management Simulation")

# Font settings
font = pygame.font.Font(None, 36)

# Traffic light system
traffic_lights = {
    'signal_1': 'RED',
    'signal_2': 'RED',
    'signal_3': 'RED',
    'signal_4': 'RED'
}

# Vehicle count per signal
vehicle_counts = {signal: random.randint(5, 50) for signal in traffic_lights.keys()}
vehicles = []

class Vehicle:
    def __init__(self, x, y, direction, signal, color):
        self.x = x
        self.y = y
        self.direction = direction
        self.signal = signal
        self.color = color
        self.speed = random.randint(3, 6)

    def can_move(self):
        return traffic_lights[self.signal] == 'GREEN'

    def move(self):
        if self.can_move():
            if self.direction == "right":
                self.x += self.speed
            elif self.direction == "left":
                self.x -= self.speed
            elif self.direction == "up":
                self.y -= self.speed
            elif self.direction == "down":
                self.y += self.speed

    def draw(self):
        pygame.draw.rect(screen, self.color, (self.x, self.y, 50, 30))
        pygame.draw.circle(screen, BLACK, (self.x + 10, self.y + 30), 5)
        pygame.draw.circle(screen, BLACK, (self.x + 40, self.y + 30), 5)

# Function to add vehicles
lanes = {
    'signal_1': (0, 200, 'right'),
    'signal_2': (screen_width, 300, 'left'),
    'signal_3': (400, screen_height, 'up'),
    'signal_4': (500, 0, 'down')
}

def add_vehicles():
    for signal, (x, y, direction) in lanes.items():
        for _ in range(vehicle_counts[signal]):
            vehicles.append(Vehicle(x, y, direction, signal, random.choice([BLUE, GREEN, YELLOW])))

def display_status():
    while True:
        screen.fill(WHITE)
        pygame.draw.rect(screen, GRAY, (0, 190, screen_width, 160))  # Horizontal road
        pygame.draw.rect(screen, GRAY, (390, 0, 160, screen_height))  # Vertical road

        for x in range(0, screen_width, 40):
            pygame.draw.rect(screen, WHITE, (x, 265, 20, 5))  # Lane markings

        for y in range(0, screen_height, 40):
            pygame.draw.rect(screen, WHITE, (465, y, 5, 20))  # Lane markings
        
        # Traffic Lights
        pole_positions = [(50, 200), (750, 300), (400, 50), (500, 500)]
        signal_names = ['Signal 1', 'Signal 2', 'Signal 3', 'Signal 4']
        
        for idx, pos in enumerate(pole_positions):
            pygame.draw.rect(screen, BLACK, (pos[0], pos[1] - 50, 10, 60))
            current_signal = f'signal_{idx + 1}'
            color = RED if traffic_lights[current_signal] == 'RED' else GREEN
            pygame.draw.circle(screen, color, (pos[0] + 5, pos[1] - 25), 10)
            text_name = font.render(signal_names[idx], True, BLACK)
            screen.blit(text_name, (pos[0] - 10, pos[1] + 20))
        
        # Display vehicle counts
        for idx, (signal, count) in enumerate(vehicle_counts.items()):
            text = font.render(f"{signal} Count: {count}", True, BLACK)
            screen.blit(text, (50, 20 + idx * 40))
        
        # Draw and move vehicles
        for vehicle in vehicles:
            vehicle.draw()
            vehicle.move()

        pygame.display.flip()
        time.sleep(0.1)

def manage_traffic():
    while True:
        highest_traffic_signal = max(vehicle_counts, key=vehicle_counts.get)
        for signal in traffic_lights.keys():
            traffic_lights[signal] = 'RED'
        traffic_lights[highest_traffic_signal] = 'GREEN'
        time.sleep(10)

def start_traffic_management():
    threading.Thread(target=manage_traffic, daemon=True).start()

def start_display_status():
    threading.Thread(target=display_status, daemon=True).start()

if __name__ == "__main__":
    add_vehicles()
    start_traffic_management()
    start_display_status()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
    pygame.quit()


In [1]:
pip install PyOpenGL PyOpenGL_accelerate

^C
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB 330.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/3.2 MB 326.8 kB/s eta 0:00:10
    --------------------------------------- 0.1/3.2 MB 393.8 kB/s eta 0:00:08
   - -------------------------------------- 0.1/3.2 MB 476.3 kB/s eta 0:00:07
   - -------------------------------------- 0.1/3.2 MB 476.3 kB/s eta 0:00:07
   - -------------------------------------- 0.1/3.2 MB 368.6 kB/s eta 0:00:09
   -- ------------------------------------- 0.2/3.2 MB 506.0 kB/s eta 0:00:06
   -- ------------------------------------- 0.2/3.2 MB 573.4 kB/s eta 0:00:06
   --- ------------------------------------ 0.3/3.2 MB 632.7 kB/s eta 0:00:05
   ---- ----------------------------------- 0.3/3.2 MB 677.0 kB/s eta 0:00:05
   ----- 

In [ ]:
import time
import threading
import random
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import pygame
from pygame.locals import *

# Initialize Pygame
pygame.init()

# Screen setup
screen_width = 1000
screen_height = 800
FPS = 60

screen = pygame.display.set_mode((screen_width, screen_height), DOUBLEBUF | OPENGL)
pygame.display.set_caption("3D Traffic Management Simulation")

# OpenGL Setup
def init_opengl():
    glutInit()  # Moved here (important)
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)  # Proper display mode
    glViewport(0, 0, screen_width, screen_height)
    glMatrixMode(GL_PROJECTION)
    gluPerspective(45, screen_width / screen_height, 0.1, 100.0)
    glMatrixMode(GL_MODELVIEW)
    glEnable(GL_DEPTH_TEST)
    glClearColor(0.8, 0.8, 0.8, 1.0)  # Light gray background

    # Basic lighting setup
    glEnable(GL_LIGHTING)
    glEnable(GL_LIGHT0)
    glLightfv(GL_LIGHT0, GL_POSITION, [0, 10, 10, 1])
    glLightfv(GL_LIGHT0, GL_DIFFUSE, [1, 1, 1, 1])
    glEnable(GL_COLOR_MATERIAL)

traffic_lights = {'signal_1': 'RED', 'signal_2': 'RED', 'signal_3': 'RED', 'signal_4': 'RED'}
vehicle_counts = {'signal_1': 0, 'signal_2': 0, 'signal_3': 0, 'signal_4': 0}
vehicles = []

class Vehicle:
    def __init__(self, x, y, z, lane, direction, signal):
        self.x, self.y, self.z = x, y, z
        self.lane, self.direction, self.signal = lane, direction, signal
        self.speed = random.uniform(0.1, 0.3)

    def can_move(self):
        return traffic_lights[self.signal] == 'GREEN'

    def move(self):
        if self.can_move():
            if self.direction == "right": self.x += self.speed
            elif self.direction == "left": self.x -= self.speed
            elif self.direction == "up": self.z -= self.speed
            elif self.direction == "down": self.z += self.speed

    def draw(self):
        glPushMatrix()
        glTranslatef(self.x, self.y, self.z)
        glColor3f(0, 0, 1)
        glBegin(GL_QUADS)
        # Bottom
        glVertex3f(-0.5, 0, -0.3); glVertex3f(0.5, 0, -0.3)
        glVertex3f(0.5, 0, 0.3); glVertex3f(-0.5, 0, 0.3)
        # Top
        glVertex3f(-0.5, 0.2, -0.3); glVertex3f(0.5, 0.2, -0.3)
        glVertex3f(0.5, 0.2, 0.3); glVertex3f(-0.5, 0.2, 0.3)
        glEnd()
        glPopMatrix()

def get_vehicle_count(signal):
    return random.randint(1, 5)  # Adjusted for visibility

def add_vehicles():
    global vehicles
    vehicles.clear()  # Clear before adding new vehicles
    lanes = {
        'signal_1': (-5, 0, 2, 'right'),
        'signal_2': (5, 0, 3, 'left'),
        'signal_3': (4, 0, 5, 'up'),
        'signal_4': (5, 0, -5, 'down')
    }
    for signal, (x, y, z, direction) in lanes.items():
        for _ in range(vehicle_counts[signal]):
            vehicles.append(Vehicle(x, y, z, signal, direction, signal))

def update_vehicle_counts():
    for signal in vehicle_counts.keys():
        vehicle_counts[signal] = get_vehicle_count(signal)
    add_vehicles()

def open_signal(signal):
    for s in traffic_lights.keys():
        traffic_lights[s] = 'RED'
    if signal != 'none':
        traffic_lights[signal] = 'GREEN'
        print(f"Signal {signal} is GREEN")

def draw_road():
    glBegin(GL_QUADS)
    glColor3f(0.5, 0.5, 0.5)
    glVertex3f(-10, -0.01, 0); glVertex3f(10, -0.01, 0)
    glVertex3f(10, -0.01, 4); glVertex3f(-10, -0.01, 4)
    glEnd()

def draw_traffic_lights():
    pole_positions = [(-3, 0, 2), (3, 0, 3), (4, 0, 3), (5, 0, -3)]
    for idx, pos in enumerate(pole_positions):
        glPushMatrix()
        glTranslatef(*pos)
        glColor3f(0, 0, 0)
        glBegin(GL_QUADS)
        glVertex3f(-0.1, 0, -0.1); glVertex3f(0.1, 0, -0.1)
        glVertex3f(0.1, 1, -0.1); glVertex3f(-0.1, 1, -0.1)
        glEnd()
        current_signal = f'signal_{idx + 1}'
        glTranslatef(0, 1.2, 0)
        glColor3f(1, 0, 0) if traffic_lights[current_signal] == 'RED' else glColor3f(0, 1, 0)
        glutSolidSphere(0.2, 10, 10)
        glPopMatrix()

def display():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    gluLookAt(0, 10, 20, 0, 0, 0, 0, 1, 0)

    draw_road()
    draw_traffic_lights()
    for vehicle in vehicles:
        vehicle.draw()
        vehicle.move()

    pygame.display.flip()

def manage_traffic():
    while True:
        update_vehicle_counts()
        highest_traffic_signal = max(vehicle_counts, key=vehicle_counts.get)
        open_signal(highest_traffic_signal)
        time.sleep(5)
        open_signal('none')
        time.sleep(3)

if __name__ == "__main__":
    init_opengl()
    glutInit()
    threading.Thread(target=manage_traffic, daemon=True).start()

    running = True
    clock = pygame.time.Clock()

    while running:
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False

        display()
        clock.tick(FPS)

    pygame.quit()


pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Signal signal_1 is GREEN


OSError: exception: access violation reading 0x00000000000000C1

Signal signal_3 is GREEN


: 

In [2]:
import time
import threading
import random
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *

# Note: We're not using GLUT anymore
# from OpenGL.GLUT import *

pygame.init()

screen_width = 1000
screen_height = 800
FPS = 60  # Lowered FPS for smoother rendering

screen = pygame.display.set_mode((screen_width, screen_height), DOUBLEBUF | OPENGL)
pygame.display.set_caption("3D Traffic Management Simulation")

font = pygame.font.Font(None, 36)

# OpenGL Setup
def init_opengl():
    glViewport(0, 0, screen_width, screen_height)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45, screen_width / screen_height, 0.1, 100.0)  # Increased far plane
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glEnable(GL_DEPTH_TEST)
    glClearColor(0.8, 0.8, 0.8, 1.0)  # Light gray background for visibility

    # Basic lighting setup
    glEnable(GL_LIGHTING)
    glEnable(GL_LIGHT0)
    glLightfv(GL_LIGHT0, GL_POSITION, [0, 10, 10, 1])  # Light from above and in front
    glLightfv(GL_LIGHT0, GL_DIFFUSE, [1, 1, 1, 1])  # White light
    glEnable(GL_COLOR_MATERIAL)  # Use glColor for material properties
    glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)

traffic_lights = {
    'signal_1': 'RED',
    'signal_2': 'RED',
    'signal_3': 'RED',
    'signal_4': 'RED'
}

vehicle_counts = {
    'signal_1': 0,
    'signal_2': 0,
    'signal_3': 0,
    'signal_4': 0
}

vehicles = []

# Helper functions to draw 3D primitives without GLUT
def draw_cube(size=1.0):
    """Draw a cube with specified size."""
    half = size / 2.0
    
    vertices = [
        # Front face
        [-half, -half, half], [half, -half, half], [half, half, half], [-half, half, half],
        # Back face
        [-half, -half, -half], [-half, half, -half], [half, half, -half], [half, -half, -half],
        # Left face
        [-half, -half, -half], [-half, -half, half], [-half, half, half], [-half, half, -half],
        # Right face
        [half, -half, -half], [half, half, -half], [half, half, half], [half, -half, half],
        # Top face
        [-half, half, -half], [-half, half, half], [half, half, half], [half, half, -half],
        # Bottom face
        [-half, -half, -half], [half, -half, -half], [half, -half, half], [-half, -half, half]
    ]
    
    glBegin(GL_QUADS)
    for i in range(0, len(vertices), 4):
        for j in range(4):
            glVertex3f(*vertices[i+j])
    glEnd()

def draw_sphere(radius=1.0, slices=10, stacks=10):
    """Draw a sphere with specified radius using manual triangles."""
    for i in range(stacks):
        lat0 = 3.14159 * (-0.5 + float(i) / stacks)
        z0 = radius * (0.5 * 3.14159) * (-0.5 + float(i) / stacks)
        
        lat1 = 3.14159 * (-0.5 + float(i+1) / stacks)
        z1 = radius * (0.5 * 3.14159) * (-0.5 + float(i+1) / stacks)
        
        glBegin(GL_QUAD_STRIP)
        for j in range(slices+1):
            lng = 2 * 3.14159 * float(j) / slices
            x = radius * (0.5 * 3.14159) * (0.5 + float(j) / slices)
            
            # First vertex
            px0 = radius * (0.5 * 3.14159) * (0.5 + float(j) / slices) * pygame.math.Vector3(
                pygame.math.Vector2(1, 0).rotate(lng * 180/3.14159)
            ).normalize().x
            py0 = radius * (0.5 * 3.14159) * (0.5 + float(j) / slices) * pygame.math.Vector3(
                pygame.math.Vector2(1, 0).rotate(lng * 180/3.14159)
            ).normalize().y
            pz0 = z0
            
            # Second vertex
            px1 = radius * (0.5 * 3.14159) * (0.5 + float(j) / slices) * pygame.math.Vector3(
                pygame.math.Vector2(1, 0).rotate(lng * 180/3.14159)
            ).normalize().x
            py1 = radius * (0.5 * 3.14159) * (0.5 + float(j) / slices) * pygame.math.Vector3(
                pygame.math.Vector2(1, 0).rotate(lng * 180/3.14159)
            ).normalize().y
            pz1 = z1
            
            glVertex3f(px0, pz0, py0)
            glVertex3f(px1, pz1, py1)
        glEnd()

class Vehicle:
    def __init__(self, x, y, z, lane, direction, signal):
        self.x = x
        self.y = y
        self.z = z
        self.lane = lane
        self.direction = direction
        self.signal = signal
        self.speed = random.uniform(0.1, 0.3)

    def can_move(self):
        return traffic_lights[self.signal] == 'GREEN'

    def move(self):
        if self.can_move():
            if self.direction == "right":
                self.x += self.speed
                if self.x > 15: 
                    return True  # Vehicle has exited the scene
            elif self.direction == "left":
                self.x -= self.speed
                if self.x < -15: 
                    return True  # Vehicle has exited the scene
            elif self.direction == "up":
                self.z -= self.speed
                if self.z < -15: 
                    return True  # Vehicle has exited the scene
            elif self.direction == "down":
                self.z += self.speed
                if self.z > 15: 
                    return True  # Vehicle has exited the scene
        return False  # Vehicle is still in the scene

    def draw(self):
        glPushMatrix()
        glTranslatef(self.x, self.y, self.z)
        
        # Determine color based on direction
        if self.direction == "right":
            glColor3f(1.0, 0.0, 0.0)  # Red for right-moving
        elif self.direction == "left":
            glColor3f(0.0, 0.0, 1.0)  # Blue for left-moving
        elif self.direction == "up":
            glColor3f(0.0, 1.0, 0.0)  # Green for up-moving
        elif self.direction == "down":
            glColor3f(1.0, 1.0, 0.0)  # Yellow for down-moving
        
        # Simple 3D car model
        glBegin(GL_QUADS)
        # Bottom
        glVertex3f(-0.5, 0, -0.3)
        glVertex3f(0.5, 0, -0.3)
        glVertex3f(0.5, 0, 0.3)
        glVertex3f(-0.5, 0, 0.3)
        # Top
        glVertex3f(-0.5, 0.2, -0.3)
        glVertex3f(0.5, 0.2, -0.3)
        glVertex3f(0.5, 0.2, 0.3)
        glVertex3f(-0.5, 0.2, 0.3)
        # Sides
        glVertex3f(-0.5, 0, -0.3)
        glVertex3f(-0.5, 0.2, -0.3)
        glVertex3f(-0.5, 0.2, 0.3)
        glVertex3f(-0.5, 0, 0.3)
        
        glVertex3f(0.5, 0, -0.3)
        glVertex3f(0.5, 0.2, -0.3)
        glVertex3f(0.5, 0.2, 0.3)
        glVertex3f(0.5, 0, 0.3)
        
        glVertex3f(-0.5, 0, -0.3)
        glVertex3f(0.5, 0, -0.3)
        glVertex3f(0.5, 0.2, -0.3)
        glVertex3f(-0.5, 0.2, -0.3)
        
        glVertex3f(-0.5, 0, 0.3)
        glVertex3f(0.5, 0, 0.3)
        glVertex3f(0.5, 0.2, 0.3)
        glVertex3f(-0.5, 0.2, 0.3)
        glEnd()
        
        # Wheels (simple black cubes)
        glColor3f(0, 0, 0)
        
        # Front left wheel
        glPushMatrix()
        glTranslatef(-0.4, 0, -0.3)
        glScalef(0.1, 0.1, 0.1)
        draw_cube()
        glPopMatrix()
        
        # Front right wheel
        glPushMatrix()
        glTranslatef(0.4, 0, -0.3)
        glScalef(0.1, 0.1, 0.1)
        draw_cube()
        glPopMatrix()
        
        # Rear left wheel
        glPushMatrix()
        glTranslatef(-0.4, 0, 0.3)
        glScalef(0.1, 0.1, 0.1)
        draw_cube()
        glPopMatrix()
        
        # Rear right wheel
        glPushMatrix()
        glTranslatef(0.4, 0, 0.3)
        glScalef(0.1, 0.1, 0.1)
        draw_cube()
        glPopMatrix()
        
        glPopMatrix()

def get_vehicle_count(signal):
    return random.randint(3, 7)  # Reduced for visibility

def add_vehicles():
    # Define lane positions based on signal
    lanes = {
        'signal_1': {'x': -15, 'y': 0, 'z': 3, 'direction': 'right'},
        'signal_2': {'x': 15, 'y': 0, 'z': 4, 'direction': 'left'},
        'signal_3': {'x': 4, 'y': 0, 'z': 15, 'direction': 'up'},
        'signal_4': {'x': 5, 'y': 0, 'z': -15, 'direction': 'down'}
    }
    
    # Create vehicles with proper spacing
    for signal, lane_info in lanes.items():
        count = vehicle_counts[signal]
        for i in range(count):
            # Add spacing between vehicles
            if lane_info['direction'] == 'right' or lane_info['direction'] == 'left':
                # Space along x-axis
                x_offset = -2 * i if lane_info['direction'] == 'right' else 2 * i
                vehicles.append(Vehicle(
                    lane_info['x'] + x_offset,
                    lane_info['y'],
                    lane_info['z'],
                    signal,
                    lane_info['direction'],
                    signal
                ))
            else:
                # Space along z-axis
                z_offset = -2 * i if lane_info['direction'] == 'up' else 2 * i
                vehicles.append(Vehicle(
                    lane_info['x'],
                    lane_info['y'],
                    lane_info['z'] + z_offset,
                    signal,
                    lane_info['direction'],
                    signal
                ))

def update_vehicle_counts():
    for signal in vehicle_counts.keys():
        vehicle_counts[signal] = get_vehicle_count(signal)

def open_signal(signal):
    for s in traffic_lights.keys():
        traffic_lights[s] = 'RED'
    if signal != 'none':
        traffic_lights[signal] = 'GREEN'
        print(f"Signal {signal} is GREEN")

def update_vehicles():
    global vehicles
    vehicles_to_remove = []
    
    # Check which vehicles need to be removed
    for i, vehicle in enumerate(vehicles):
        if vehicle.move():  # If vehicle has exited the scene
            vehicles_to_remove.append(i)
    
    # Remove vehicles that have exited (in reverse order to avoid index issues)
    for i in sorted(vehicles_to_remove, reverse=True):
        if i < len(vehicles):  # Safety check
            vehicles.pop(i)

def draw_road():
    # Disable lighting for the road
    glDisable(GL_LIGHTING)
    
    # Draw the ground plane
    glBegin(GL_QUADS)
    glColor3f(0.3, 0.3, 0.3)  # Darker gray for ground
    glVertex3f(-30, -0.1, -30)
    glVertex3f(30, -0.1, -30)
    glVertex3f(30, -0.1, 30)
    glVertex3f(-30, -0.1, 30)
    glEnd()
    
    # Draw horizontal road
    glBegin(GL_QUADS)
    glColor3f(0.5, 0.5, 0.5)  # Gray for road
    glVertex3f(-30, -0.05, 2)
    glVertex3f(30, -0.05, 2)
    glVertex3f(30, -0.05, 6)
    glVertex3f(-30, -0.05, 6)
    glEnd()
    
    # Draw vertical road
    glBegin(GL_QUADS)
    glColor3f(0.5, 0.5, 0.5)  # Gray for road
    glVertex3f(2, -0.05, -30)
    glVertex3f(6, -0.05, -30)
    glVertex3f(6, -0.05, 30)
    glVertex3f(2, -0.05, 30)
    glEnd()
    
    # Draw road markings (yellow center lines)
    glBegin(GL_QUADS)
    glColor3f(1.0, 1.0, 0.0)  # Yellow
    # Horizontal road center line
    glVertex3f(-30, -0.04, 3.9)
    glVertex3f(30, -0.04, 3.9)
    glVertex3f(30, -0.04, 4.1)
    glVertex3f(-30, -0.04, 4.1)
    # Vertical road center line
    glVertex3f(3.9, -0.04, -30)
    glVertex3f(4.1, -0.04, -30)
    glVertex3f(4.1, -0.04, 30)
    glVertex3f(3.9, -0.04, 30)
    glEnd()
    
    # Re-enable lighting
    glEnable(GL_LIGHTING)

def draw_traffic_lights():
    # Positions for traffic light poles
    pole_positions = [
        {'x': -2, 'y': 0, 'z': 2, 'signal': 'signal_1'},   # For right-moving traffic
        {'x': 6, 'y': 0, 'z': 6, 'signal': 'signal_2'},    # For left-moving traffic
        {'x': 2, 'y': 0, 'z': 6, 'signal': 'signal_3'},    # For up-moving traffic
        {'x': 6, 'y': 0, 'z': 2, 'signal': 'signal_4'}     # For down-moving traffic
    ]
    
    for pos in pole_positions:
        signal = pos['signal']
        
        glPushMatrix()
        glTranslatef(pos['x'], pos['y'], pos['z'])
        
        # Draw pole
        glColor3f(0.3, 0.3, 0.3)  # Dark gray pole
        glPushMatrix()
        glTranslatef(0, 0.5, 0)  # Center of pole
        glScalef(0.2, 1.0, 0.2)  # Pole dimensions
        draw_cube()  # Use our custom cube function instead of glutSolidCube
        glPopMatrix()
        
        # Draw traffic light box
        glPushMatrix()
        glTranslatef(0, 1.2, 0)  # Position at top of pole
        glColor3f(0.2, 0.2, 0.2)  # Black box
        glScalef(0.4, 0.6, 0.3)  # Box dimensions
        draw_cube()  # Use our custom cube function
        glPopMatrix()
        
        # Draw signal light
        glPushMatrix()
        glTranslatef(0, 1.2, 0.1)  # Position light at front of box
        
        # Set light color based on traffic light state
        if traffic_lights[signal] == 'RED':
            glColor3f(1.0, 0.0, 0.0)  # Red
        else:
            glColor3f(0.0, 1.0, 0.0)  # Green
            
        # Use a simpler sphere representation
        glScalef(0.15, 0.15, 0.15)
        draw_sphere()  # Use our custom sphere function instead of glutSolidSphere
        glPopMatrix()
        
        glPopMatrix()

def display_status():
    clock = pygame.time.Clock()
    
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                return

        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glLoadIdentity()
        
        # Set camera position and orientation
        gluLookAt(0, 15, 20,  # Camera position: higher and farther back
                 0, 0, 0,     # Look at center
                 0, 1, 0)     # Up direction
        
        # Draw scene elements
        draw_road()
        draw_traffic_lights()
        
        # Draw all vehicles
        for vehicle in vehicles:
            vehicle.draw()
        
        # Update vehicle positions
        update_vehicles()
        
        pygame.display.flip()
        clock.tick(FPS)

def manage_traffic():
    while True:
        # Update traffic counts
        update_vehicle_counts()
        
        # Add new vehicles based on counts
        add_vehicles()
        
        # Find signal with highest traffic
        highest_traffic_signal = max(vehicle_counts, key=vehicle_counts.get)
        
        # Open signal with highest traffic
        open_signal(highest_traffic_signal)
        
        # Green light duration
        time.sleep(10)
        
        # All red transition
        open_signal('none')
        
        # Red light duration
        time.sleep(5)

if __name__ == "__main__":
    # Initialize OpenGL
    init_opengl()
    
    # Start traffic management thread
    traffic_thread = threading.Thread(target=manage_traffic, daemon=True)
    traffic_thread.start()
    
    # Run display in main thread for better event handling
    display_status()

Signal signal_3 is GREEN


ValueError: Vector3 must be set with 3 real numbers, a sequence of 3 real numbers, or another Vector3 instance

Signal signal_2 is GREEN
